In [1]:
# setting
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer                          

from sklearn.compose import ColumnTransformer,make_column_transformer

from sklearn.decomposition import PCA



In [2]:
work_dir = '/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/'

train = pd.read_csv((work_dir + 'PJT002_train.csv'),encoding='utf-8' )
validation = pd.read_csv((work_dir + 'PJT002_validation .csv'),encoding='utf-8' )
test = pd.read_csv((work_dir + '/' + 'PJT002_test.csv'),encoding='utf-8')

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1단계

### Date 변수 처리

In [3]:
def dt_of_fr_transform(df) :
    year_list = []
    month_list =  []
    day_list = []
    weekday_list = []
    hour_list = []
    
    season_list = []
    
    for i in range(len(df)) : 
        date_0=df.dt_of_fr[i].split(' ')[0] # '2017-10-20'
        time_0=df.dt_of_fr[i].split(' ')[1] # '05:54:00'
        
        year = int(date_0.split('-')[0]) # 2017
        month = int(date_0.split('-')[1]) # 10
        day = int(date_0.split('-')[2]) # 20
        weekday = datetime.date(year,month,day).weekday() # 0 : 월~ 6 : 일
        
        hour = int(time_0.split(':')[0]) # 05
        
        
        if month in [3,4,5] :
            season = 0
        elif month in [6,7,8] :
            season = 1
        elif month in [9,10,11] :
            season = 2 
        else :
            season =3
            
        year_list.append(year)
        month_list.append(month)
        day_list.append(day)
        weekday_list.append(weekday)
        season_list.append(season)
        
        hour_list.append(hour)
        
    df['year'] = year_list
    df['month'] = month_list
    df['day'] = day_list
    df['weekday'] = weekday_list
    df['season'] = season_list
    df['hour'] = hour_list
    # 다른 분들이 dt_of_fr 쓰실지 몰라서 일단은 drop 보류.
    # df = df.drop(['dt_of_fr'],axis=1)
    return df

In [4]:
# 함수 적용
train = dt_of_fr_transform(train)
test = dt_of_fr_transform(test)
validation = dt_of_fr_transform(validation)

### fr_yn 변수 처리

In [5]:
train['fr_yn'] = pd.get_dummies(train['fr_yn'])['Y']
validation['fr_yn'] = pd.get_dummies(validation['fr_yn'])['Y']

### 'rgnl_ar_nm','rgnl_ar_nm2','jmk','lnd_us_sttn_nm','rd_sd_nm'

In [6]:
def land_transform_step_1(df) :
    # rgnl_ar_nm, rgnl_ar_nm, jmk,lnd_us_sttn_nm,rd_sd_nm 행의 NA 값들을 모두 'blank'라는 새로운 범주로 지정해줌
    # (참고로 rgnl_ar_nm 이 NA인 경우 모두 rgnl_ar_nm2 값 또한 NA)
    values = {'rgnl_ar_nm': 'blank', 'rgnl_ar_nm2': 'blank','jmk':'blank','lnd_us_sttn_nm':'blank','rd_sd_nm':'blank'}
    df = df.fillna(value=values)
    
    # rgnl_ar_nm2 처리1 : rgnl_ar_nm2가 지정되지 않은 경우 그 행의 rgnl_ar_nm1의 값을 따르게 함.
    ix = df[ df['rgnl_ar_nm2']=='지정되지않음' ].index
    df.loc[ix,'rgnl_ar_nm2'] = df.loc[ix,'rgnl_ar_nm']
    return df

In [7]:
train = land_transform_step_1(train)
test = land_transform_step_1(test)
validation = land_transform_step_1(validation)

In [8]:
# train, test, validation set마다 포함하고 있는 범주가 조금씩 달라서 한번에 묶어서 기준 세움

# rgnl_ar_nm, rgnl_ar_nm2 변수의 경우 rgnl_ar_nm_uniques 가지고 fitting하도록 함.
rgnl_ar_nm_uniques = train.rgnl_ar_nm.append(train.rgnl_ar_nm2).append(test.rgnl_ar_nm).append(test.rgnl_ar_nm2).append(validation.rgnl_ar_nm).append(validation.rgnl_ar_nm2).unique()
rgnl_ar_nm_uniques = rgnl_ar_nm_uniques.reshape(-1,1)

# 'jmk','lnd_us_sttn_nm','rd_sd_nm' 또한 마찬가지
jmk_uniques = train.jmk.append(test.jmk).append(validation.jmk).unique()
jmk_uniques = jmk_uniques.reshape(-1,1)

lnd_us_sttn_nm_uniques = train.lnd_us_sttn_nm.append(test.lnd_us_sttn_nm).append(validation.lnd_us_sttn_nm).unique()
lnd_us_sttn_nm_uniques = lnd_us_sttn_nm_uniques.reshape(-1,1)

rd_sd_nm_uniques = train.rd_sd_nm.append(test.rd_sd_nm).append(validation.rd_sd_nm).unique()
rd_sd_nm_uniques = rd_sd_nm_uniques.reshape(-1,1)

'''
print(rgnl_ar_nm_uniques)
print(jmk_uniques)
print(lnd_us_sttn_nm_uniques)
print(rd_sd_nm_uniques)
'''

'\nprint(rgnl_ar_nm_uniques)\nprint(jmk_uniques)\nprint(lnd_us_sttn_nm_uniques)\nprint(rd_sd_nm_uniques)\n'

In [9]:
def land_transform_step_2(df):
    #################################################################################
    # rgnl_ar_nm, rgnl_ar_nm2 처리 
    # train, test, validation set마다 포함하고 있는 범주가 조금씩 달라서 한번에 묶어서 기준 세움
    
    oh_enc_rgnl_ar_nm = OneHotEncoder()
    oh_enc_rgnl_ar_nm.fit(rgnl_ar_nm_uniques)
    
    # category 이름 list
    cat_list_rgnl = oh_enc_rgnl_ar_nm.__dict__['categories_'][0].tolist()
    
    # encoder 들어갈 수 있게 모양 잡아줌.
    ar1 = np.array(df.rgnl_ar_nm).reshape(-1,1)
    ar2 = np.array(df.rgnl_ar_nm2).reshape(-1,1)
    
    # onehot encoding
    ar1_onehot = oh_enc_rgnl_ar_nm.transform(ar1)
    ar2_onehot = oh_enc_rgnl_ar_nm.transform(ar2)
    
    # 더해주고 2로 나누고 본 데이터에 붙이고 rgnl_ar_nm, rgnl_ar_nm2 drop
    onehot = (ar1_onehot/2+ar2_onehot/2).toarray()
    df = df.drop(['rgnl_ar_nm','rgnl_ar_nm2'],axis=1)
    df = pd.concat([df,pd.DataFrame(onehot,columns=cat_list_rgnl)],axis=1)
    
    #################################################################################
    # jmk, lnd_us_sttn_nm, rd_sd_nm 처리
    # 위와 같은 방법
    
    # jmk
    oh_enc_jmk = OneHotEncoder()
    oh_enc_jmk.fit(jmk_uniques)
    
    cat_list_jmk = oh_enc_jmk.__dict__['categories_'][0].tolist()
    jmk_arr = np.array(df.jmk).reshape(-1,1)
    jmk_onehot = oh_enc_jmk.transform(jmk_arr).toarray()
    df = df.drop(['jmk'],axis=1)
    df = pd.concat([df,pd.DataFrame(jmk_onehot,columns=cat_list_jmk)],axis=1)
    
    #lnd_us_sttn_nm
    oh_enc_lnd_us_sttn_nm = OneHotEncoder()
    oh_enc_lnd_us_sttn_nm.fit(lnd_us_sttn_nm_uniques)

    cat_list_lnd_us_sttn_nm = oh_enc_lnd_us_sttn_nm.__dict__['categories_'][0].tolist()
    lnd_us_sttn_nm_arr = np.array(df.lnd_us_sttn_nm).reshape(-1,1)
    lnd_us_sttn_nm_onehot = oh_enc_lnd_us_sttn_nm.transform(lnd_us_sttn_nm_arr).toarray()
    df = df.drop(['lnd_us_sttn_nm'],axis=1)
    df = pd.concat([df,pd.DataFrame(lnd_us_sttn_nm_onehot,columns=cat_list_lnd_us_sttn_nm)],axis=1)
    
    #rd_sd_nm
    oh_enc_rd_sd_nm = OneHotEncoder()
    oh_enc_rd_sd_nm.fit(rd_sd_nm_uniques)
    
    cat_list_rd_sd_nm = oh_enc_rd_sd_nm.__dict__['categories_'][0].tolist()
    rd_sd_nm_arr = np.array(df.rd_sd_nm).reshape(-1,1)
    rd_sd_nm_onehot = oh_enc_rd_sd_nm.transform(rd_sd_nm_arr).toarray()
    df = df.drop(['rd_sd_nm'],axis=1)
    df = pd.concat([df,pd.DataFrame(rd_sd_nm_onehot,columns=cat_list_rd_sd_nm)],axis=1)

    return df
    
    
    

In [10]:
train = land_transform_step_2(train)
test= land_transform_step_2(test)
validation = land_transform_step_2(validation)

## 지현님 파트(1단계)

* 건물 승인일자 년 단위로 자르기 함수 (소수점 포함 10자리인 글자만 찾아 자르기)

* 그 후, 2019에서 뺀 값으로 대체

In [11]:
def cut_func(x):
    if len(str(x)) == 10:
        return str(x)[:4]

In [12]:
train['dt_of_athrztn'] = train['dt_of_athrztn'].apply(cut_func)
test['dt_of_athrztn'] = test['dt_of_athrztn'].apply(cut_func)
validation['dt_of_athrztn'] = validation['dt_of_athrztn'].apply(cut_func)

In [13]:
train['dt_of_athrztn'] = pd.to_numeric(train['dt_of_athrztn'])
test['dt_of_athrztn'] = pd.to_numeric(test['dt_of_athrztn'])
validation['dt_of_athrztn'] = pd.to_numeric(validation['dt_of_athrztn'])

In [14]:
train['dt_of_athrztn'] = 2019 - train['dt_of_athrztn']
test['dt_of_athrztn'] = 2019 - test['dt_of_athrztn']
validation['dt_of_athrztn'] = 2019 - validation['dt_of_athrztn']

* 지상층, 지하층, 건물채수 카테고리화

In [15]:
train_data = [train, test, validation]

In [16]:
# 지상층 카테고리화

for dataset in train_data:
    dataset.loc[dataset['ttl_grnd_flr'] == 0, 'ttl_grnd_flr'] = 0,
    dataset.loc[(dataset['ttl_grnd_flr'] >= 1) &  (dataset['ttl_grnd_flr'] <= 10), 'ttl_grnd_flr'] = 10, 
    dataset.loc[(dataset['ttl_grnd_flr'] > 10) &  (dataset['ttl_grnd_flr'] <= 20), 'ttl_grnd_flr'] = 20,
    dataset.loc[(dataset['ttl_grnd_flr'] > 20) &  (dataset['ttl_grnd_flr'] <= 30), 'ttl_grnd_flr'] = 30,
    dataset.loc[(dataset['ttl_grnd_flr'] > 30) &  (dataset['ttl_grnd_flr'] <= 40), 'ttl_grnd_flr'] = 40,
    dataset.loc[(dataset['ttl_grnd_flr'] > 40) &  (dataset['ttl_grnd_flr'] <= 55), 'ttl_grnd_flr'] = 55,
    dataset.loc[dataset['ttl_grnd_flr'] > 55, 'ttl_grnd_flr'] = 56

In [17]:
# 지하층 카테고리화

for dataset in train_data:
    dataset.loc[dataset['ttl_dwn_flr'] == 0, 'ttl_dwn_flr'] = 0,
    dataset.loc[(dataset['ttl_dwn_flr'] >= 1) &  (dataset['ttl_dwn_flr'] <= 4), 'ttl_dwn_flr'] = 4, 
    dataset.loc[(dataset['ttl_dwn_flr'] > 4) &  (dataset['ttl_dwn_flr'] <= 10), 'ttl_dwn_flr'] = 10,
    dataset.loc[(dataset['ttl_dwn_flr'] > 10) &  (dataset['ttl_dwn_flr'] <= 20), 'ttl_dwn_flr'] = 20,
    dataset.loc[dataset['ttl_dwn_flr'] > 20, 'ttl_dwn_flr'] = 21


In [18]:
# 건물 채수 카테고리화

for dataset in train_data:
    dataset.loc[dataset['bldng_cnt'] == 0, 'bldng_cnt'] = 0,
    dataset.loc[(dataset['bldng_cnt'] >= 1) &  (dataset['bldng_cnt'] <= 5), 'bldng_cnt'] = 5, 
    dataset.loc[(dataset['bldng_cnt'] > 5) &  (dataset['bldng_cnt'] <= 10), 'bldng_cnt'] = 10,
    dataset.loc[(dataset['bldng_cnt'] > 10) &  (dataset['bldng_cnt'] <= 20), 'bldng_cnt'] = 20,
    dataset.loc[(dataset['bldng_cnt'] > 20) &  (dataset['bldng_cnt'] <= 30), 'bldng_cnt'] = 30,
    dataset.loc[dataset['bldng_cnt'] > 30, 'bldng_cnt'] = 31

----------------------------

In [19]:
# set the NA values to 'blank' and -999

train[['bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn']] = train[['bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn']].fillna('blank')
test[['bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn']] = test[['bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn']].fillna('blank')
validation[['bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn']] = validation[['bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn']].fillna('blank')

train[['ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt']] = train[['ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt']].fillna(-999)
test[['ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt']] = test[['ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt']].fillna(-999)
validation[['ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt']] = validation[['ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt']].fillna(-999)

    

In [20]:
# 'bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn','ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt'
# train, test, validation에서 각각 가지고 있는 총 카테고리의 수가 달라서 묶어서 한번에 피팅해주기로 함.

bldng_us_uniques = train.bldng_us.append(train.bldng_us).append(test.bldng_us).append(test.bldng_us).append(validation.bldng_us).append(validation.bldng_us).unique()
bldng_us_uniques = bldng_us_uniques.reshape(-1,1)

bldng_archtctr_uniques = train.bldng_archtctr.append(train.bldng_archtctr).append(test.bldng_archtctr).append(test.bldng_archtctr).append(validation.bldng_archtctr).append(validation.bldng_archtctr).unique()
bldng_archtctr_uniques = bldng_archtctr_uniques.reshape(-1,1)

bldng_us_clssfctn_uniques = train.bldng_us_clssfctn.append(train.bldng_us_clssfctn).append(test.bldng_us_clssfctn).append(test.bldng_us_clssfctn).append(validation.bldng_us_clssfctn).append(validation.bldng_us_clssfctn).unique()
bldng_us_clssfctn_uniques = bldng_us_clssfctn_uniques.reshape(-1,1)

ttl_grnd_flr_uniques = train.ttl_grnd_flr.append(train.ttl_grnd_flr).append(test.ttl_grnd_flr).append(test.ttl_grnd_flr).append(validation.ttl_grnd_flr).append(validation.ttl_grnd_flr).unique()
ttl_grnd_flr_uniques = ttl_grnd_flr_uniques.reshape(-1,1)

ttl_dwn_flr_uniques = train.ttl_dwn_flr.append(train.ttl_dwn_flr).append(test.ttl_dwn_flr).append(test.ttl_dwn_flr).append(validation.ttl_dwn_flr).append(validation.ttl_dwn_flr).unique()
ttl_dwn_flr_uniques = ttl_dwn_flr_uniques.reshape(-1,1)

bldng_cnt_uniques = train.bldng_cnt.append(train.bldng_cnt).append(test.bldng_cnt).append(test.bldng_cnt).append(validation.bldng_cnt).append(validation.bldng_cnt).unique()
bldng_cnt_uniques = bldng_cnt_uniques.reshape(-1,1)


In [21]:
def bldng_transform(df):
 #################################################################################
    # 'bldng_us', 'bldng_archtctr', 'bldng_us_clssfctn','ttl_grnd_flr', 'ttl_dwn_flr', 'bldng_cnt' 처리
    # 위와 같은 방법
    
    # bldng_us
    oh_enc_bldng_us = OneHotEncoder()
    oh_enc_bldng_us.fit(bldng_us_uniques)
    
    cat_list_bldng_us = oh_enc_bldng_us.__dict__['categories_'][0].tolist()
    bldng_us_arr = np.array(df.bldng_us).reshape(-1,1)
    bldng_us_onehot = oh_enc_bldng_us.transform(bldng_us_arr).toarray()
    df = df.drop(['bldng_us'],axis=1)
    df = pd.concat([df,pd.DataFrame(bldng_us_onehot,columns=cat_list_bldng_us)],axis=1)
        
    # bldng_archtctr
    oh_enc_bldng_archtctr = OneHotEncoder()
    oh_enc_bldng_archtctr.fit(bldng_archtctr_uniques)
    
    cat_list_bldng_archtctr = oh_enc_bldng_archtctr.__dict__['categories_'][0].tolist()
    bldng_archtctr_arr = np.array(df.bldng_archtctr).reshape(-1,1)
    bldng_archtctr_onehot = oh_enc_bldng_archtctr.transform(bldng_archtctr_arr).toarray()
    df = df.drop(['bldng_archtctr'],axis=1)
    df = pd.concat([df,pd.DataFrame(bldng_archtctr_onehot,columns=cat_list_bldng_archtctr)],axis=1)
    
    # bldng_us_clssfctn
    oh_enc_bldng_us_clssfctn = OneHotEncoder()
    oh_enc_bldng_us_clssfctn.fit(bldng_us_clssfctn_uniques)
    
    cat_list_bldng_us_clssfctn = oh_enc_bldng_us_clssfctn.__dict__['categories_'][0].tolist()
    bldng_us_clssfctn_arr = np.array(df.bldng_us_clssfctn).reshape(-1,1)
    bldng_us_clssfctn_onehot = oh_enc_bldng_us_clssfctn.transform(bldng_us_clssfctn_arr).toarray()
    df = df.drop(['bldng_us_clssfctn'],axis=1)
    df = pd.concat([df,pd.DataFrame(bldng_us_clssfctn_onehot,columns=cat_list_bldng_us_clssfctn)],axis=1)
    
    # ttl_grnd_flr
    oh_enc_ttl_grnd_flr = OneHotEncoder(categories='auto' )
    oh_enc_ttl_grnd_flr.fit(ttl_grnd_flr_uniques)
    
    cat_list_ttl_grnd_flr = oh_enc_ttl_grnd_flr.__dict__['categories_'][0].tolist()
    ttl_grnd_flr_arr = np.array(df.ttl_grnd_flr).reshape(-1,1)
    ttl_grnd_flr_onehot = oh_enc_ttl_grnd_flr.transform(ttl_grnd_flr_arr).toarray()
    df = df.drop(['ttl_grnd_flr'],axis=1)
    df = pd.concat([df,pd.DataFrame(ttl_grnd_flr_onehot,columns=cat_list_ttl_grnd_flr)],axis=1)
    
    # ttl_dwn_flr
    oh_enc_ttl_dwn_flr = OneHotEncoder(categories='auto' )
    oh_enc_ttl_dwn_flr.fit(ttl_dwn_flr_uniques)
    
    cat_list_ttl_dwn_flr = oh_enc_ttl_dwn_flr.__dict__['categories_'][0].tolist()
    ttl_dwn_flr_arr = np.array(df.ttl_dwn_flr).reshape(-1,1)
    ttl_dwn_flr_onehot = oh_enc_ttl_dwn_flr.transform(ttl_dwn_flr_arr).toarray()
    df = df.drop(['ttl_dwn_flr'],axis=1)
    df = pd.concat([df,pd.DataFrame(ttl_dwn_flr_onehot,columns=cat_list_ttl_dwn_flr)],axis=1)
    
    # bldng_cnt
    oh_enc_bldng_cnt = OneHotEncoder(categories='auto' )
    oh_enc_bldng_cnt.fit(bldng_cnt_uniques)
    
    cat_list_bldng_cnt = oh_enc_bldng_cnt.__dict__['categories_'][0].tolist()
    bldng_cnt_arr = np.array(df.bldng_cnt).reshape(-1,1)
    bldng_cnt_onehot = oh_enc_bldng_cnt.transform(bldng_cnt_arr).toarray()
    df = df.drop(['bldng_cnt'],axis=1)
    df = pd.concat([df,pd.DataFrame(bldng_cnt_onehot,columns=cat_list_bldng_cnt)],axis=1)
    
    return df
    
    
    

In [22]:
train = bldng_transform(train)
test = bldng_transform(test)
validation = bldng_transform(validation)

### 성호님 파트

* 도시 시골 변수 추가

In [99]:
train['emd_nm'] = train['emd_nm'].fillna(value='blank')
train.loc[train['emd_nm'].str[5:8].str.endswith('시'), 'urban'] = 'Y'
train.loc[train['emd_nm'].str[5:8].str.endswith('군'), 'urban'] = 'N'


test['emd_nm'] = test['emd_nm'].fillna(value='blank')
test.loc[test['emd_nm'].str.endswith('동'), 'urban'] = 'Y'
test.loc[test['emd_nm'].str.endswith('면'), 'urban'] = 'N'
test.loc[test['emd_nm'].str.endswith('읍'), 'urban'] = 'N'


validation['emd_nm'] = validation['emd_nm'].fillna(value='blank')
validation.loc[validation['emd_nm'].str.endswith('동'), 'urban'] = 'Y'
validation.loc[validation['emd_nm'].str.endswith('면'), 'urban'] = 'N'
validation.loc[validation['emd_nm'].str.endswith('읍'), 'urban'] = 'N'
#validation.loc[validation['emd_nm'].str.endswith('k'), 'urban'] = 'blank'


In [101]:
train = pd.get_dummies(train, prefix=['urban'],drop_first=False,columns=['urban'])
test = pd.get_dummies(test, prefix=['urban'],drop_first=False,columns=['urban'])
validation =pd.get_dummies(validation, prefix=['urban'],drop_first=False,columns=['urban'])

# 2단계

## ColumnTransformer 위한 변수 정의

In [103]:
weather_var_1 = ['prcpttn']
weather_var_2 = ['tmprtr','wnd_spd','wnd_drctn','hmdt'] 
weather_var = weather_var_1 + weather_var_2

eg_var = train.loc[:,'gas_engry_us_201401':'ele_engry_us_201812'].keys().tolist()
lw_var = train.loc[:,'lw_13101010':'lw_13141011'] .keys().tolist()
egl_var = eg_var + lw_var # 모든 관심변수들

building_numeric_feature = ['bldng_ar', 'ttl_ar', 'lnd_ar']
building_date = ['dt_of_athrztn']


urban_feature = ['mlt_us_yn']
numerical_feature = ['hm_cnt', 'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc', 'fr_mn_cnt', 
                      'cctv_dstnc', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc',
                      'ahsm_dstnc', 'no_tbc_zn_dstnc', 'bldng_cnt_in_50m']
price_feature = ['bldng_ar_prc']

drop_var =['dt_of_fr','fr_yn','id','fr_wthr_fclt_in_100m', 'emd_nm','cctv_in_100m',
            'trgt_crtr', 'fr_fghtng_fclt_spcl_css_5_yn', 'fr_fghtng_fclt_spcl_css_6_yn', 
           'us_yn','dngrs_thng_yn', 'slf_fr_brgd_yn', 'blk_dngrs_thng_mnfctr_yn', 'cltrl_hrtg_yn', 
           'year', 'month', 'day', 'weekday', 'season', 'hour',]



select_var = weather_var_1 + weather_var_2 + eg_var + lw_var + building_numeric_feature + building_date + urban_feature + numerical_feature + price_feature


# 'fr_sttn_dstnc' - na : 0 , numeric, 
# 'bldng_ar_prc' - na : 21895, numeric,
# 'fr_wthr_fclt_dstnc' - na : 0, numeric
# 'fr_mn_cnt' - na : 41 , numeric
# 'mlt_us_yn' - na : 0 , category w 2
# 'cctv_dstnc' - na : 0 , numeric
# 'fr_wthr_fclt_in_100m' - na : 0, numeric(discrete)
# 'cctv_in_100m'  - na: 0, numeric(discrete)
# 'tbc_rtl_str_dstnc' - na : 0 , numeric
# 'sft_emrgnc_bll_dstnc' - na : 0, numeric
# 'ahsm_dstnc' - na : 0 , numeric
#  'no_tbc_zn_dstnc' - na : 0, numeric
# 'bldng_cnt_in_50m' - na : 0 , numeric(highly skewed to the left)
# 'trgt_crtr' - na : 51022, cat
# 'fr_fghtng_fclt_spcl_css_5_yn' - na : 50762, category w 1(N - 8437개만 존재 )
# 'us_yn' - na : 49447, category w 3( Y and N and T) * T는 Y의 오타
# 'dngrs_thng_yn' - na : 49447, cate w 2( N and Y)
# 


In [104]:

train_2 = train.drop(drop_var,axis=1)
test_2 = test.drop(drop_var,axis=1)
validation_2 = validation.drop(drop_var,axis=1)


In [105]:
list(train_2.keys())

['bldng_ar',
 'ttl_ar',
 'lnd_ar',
 'dt_of_athrztn',
 'tmprtr',
 'prcpttn',
 'wnd_spd',
 'wnd_drctn',
 'hmdt',
 'gas_engry_us_201401',
 'ele_engry_us_201401',
 'gas_engry_us_201402',
 'ele_engry_us_201402',
 'gas_engry_us_201403',
 'ele_engry_us_201403',
 'gas_engry_us_201404',
 'ele_engry_us_201404',
 'gas_engry_us_201405',
 'ele_engry_us_201405',
 'gas_engry_us_201406',
 'ele_engry_us_201406',
 'gas_engry_us_201407',
 'ele_engry_us_201407',
 'gas_engry_us_201408',
 'ele_engry_us_201408',
 'gas_engry_us_201409',
 'ele_engry_us_201409',
 'gas_engry_us_201410',
 'ele_engry_us_201410',
 'gas_engry_us_201411',
 'ele_engry_us_201411',
 'gas_engry_us_201412',
 'ele_engry_us_201412',
 'gas_engry_us_201501',
 'ele_engry_us_201501',
 'gas_engry_us_201502',
 'ele_engry_us_201502',
 'gas_engry_us_201503',
 'ele_engry_us_201503',
 'gas_engry_us_201504',
 'ele_engry_us_201504',
 'gas_engry_us_201505',
 'ele_engry_us_201505',
 'gas_engry_us_201506',
 'ele_engry_us_201506',
 'gas_engry_us_201507',
 

In [107]:
train_2.iloc[:,155]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
59194    0.0
59195    0.0
59196    0.0
59197    0.0
59198    0.0
Name: blank, Length: 59199, dtype: float64

In [108]:
train_3 = np.array(train_2.iloc[:,155:])
test_3 = np.array(test_2.iloc[:,155:])
validation_3 = np.array(validation_2.iloc[:,155:])

## Weather Pipeline

##### 'tmprtr','prcpttn','wnd_spd','wnd_drctn','hmdt'

In [109]:
weather_imputer = ColumnTransformer([
    ('prcpttn_imputer',SimpleImputer(strategy='constant',fill_value=-1),weather_var_1),
    ('otehrs_imputer',IterativeImputer(),weather_var_2)
     ])

weather_scale_PCA = Pipeline([
    ('Scaler',StandardScaler()),
    ('PCA',PCA())
])

weather_pipe = Pipeline([
    ('weather_imputer',weather_imputer),
    ('weather_scale_PCA',weather_scale_PCA)
    
])

## Gas, Ele, Lw Pipeline


###  불이 난 달의 Gas와 Ele 사용량을 열으로 추가하는 변환기를 정의합니다. 



In [110]:
# Train

class TrFmGEAdder(BaseEstimator, TransformerMixin):
  def __init__(self, index = eg_var):
    self.index = ['dt_of_fr'] + index
  
  def fit(self, X,y = None):
    return(self)
  
  def transform(self, X, y = None):
    index = self.index
    X = np.append(np.array(train['dt_of_fr']).reshape(-1,1), X, axis = 1) # X를 받아서 앞에 dt_of_fr열을 추가합니다.
    X = pd.DataFrame(X) # 차후 열 이름을 기준으로 사용해야 하기 때문에 Pandas로 변환합니다. 
    X.columns = train[['dt_of_fr']+eg_var].keys() # 열 이름을 추가합니다. eg_var 대상으로 진행할 것이기 때문에 이렇게 씁니다.

    train_rst = X.assign(
        fr_month_gas  = 'gas'+X['dt_of_fr'].str.slice_replace(start=7, stop=20, repl='').str.replace(pat='-', repl='', regex=False)
        ).assign(
            fr_month_ele  = 'ele'+X['dt_of_fr'].str.slice_replace(start=7, stop=20, repl='').str.replace(pat='-', repl='', regex=False)
            ) # gas/ele + 불이 난 달을 값으로 갖는 열을 생성합니다. 이걸 생성하지 않고 하는 것도 할 수 있을거 같은데 지금은 뇌가 멈췄으니 일단 만듭니다.
    
    tk = train_rst.keys()[np.where(pd.Series(train_rst.keys()) == index[0])[0][0]:np.where(pd.Series(train_rst.keys()) == index[-1])[0][0]+1]
    keys = pd.Series(tk).str.split('_', expand = True).iloc[:,[0,3]]
    keys[1] = keys[0]+keys[3]
    keys[1][0] = 'dt_of_fr' # 변경된 열 이름의 리스트입니다. 가장 앞의 값이 누락되어서 추가해주었습니다.
    
    coln = keys[1].ravel().tolist()+['fr_month_gas','fr_month_ele'] #열 이름을 바꿔줄 준비. 앞선 fr_month_gas/ ele열을 index로 사용하기 위함입니다
    train_rst.columns = coln #열 이름 변경합니다.

    fm_gas_index = train_rst.fr_month_gas.map(lambda x: np.where(x == train_rst.keys())[0][0]) # 불이 난 월의 gas의 열 index
    fm_ele_index = train_rst.fr_month_ele.map(lambda x: np.where(x == train_rst.keys())[0][0]) # 불이 난 월의 ele의 열 index

    train_eg = train_rst.assign(
        gas_fm = np.array(train_rst)[np.arange(train.shape[0]),fm_gas_index.values] # 최종적으로 두 index를 기준으로 값을 대입합니다.
        ).assign(
            ele_fm = np.array(train_rst)[np.arange(train.shape[0]),fm_ele_index.values]
            )[['gas_fm','ele_fm']] # 이 두 열만 남기거나
            #.drop(['fr_month_gas','fr_month_ele','dt_of_fr'], axis = 1) # 이 두 열을 더하거나. 일단은 두 열만 남겼습니다. 
    
    return np.array(train_eg)

In [111]:
#  Val. 

class VFmGEAdder(BaseEstimator, TransformerMixin):
  def __init__(self, index = eg_var):
    self.index = ['dt_of_fr'] + index
  
  def fit(self, X,y = None):
    return(self)
  
  def transform(self, X, y = None):
    index = self.index
    X = np.append(np.array(validation['dt_of_fr']).reshape(-1,1), X, axis = 1) # X를 받아서 앞에 dt_of_fr열을 추가합니다.
    X = pd.DataFrame(X) # 차후 열 이름을 기준으로 사용해야 하기 때문에 Pandas로 변환합니다. 
    X.columns = validation[['dt_of_fr']+eg_var].keys() # 열 이름을 추가합니다. eg_var 대상으로 진행할 것이기 때문에 이렇게 씁니다.

    train_rst = X.assign(
        fr_month_gas  = 'gas'+X['dt_of_fr'].str.slice_replace(start=7, stop=20, repl='').str.replace(pat='-', repl='', regex=False)
        ).assign(
            fr_month_ele  = 'ele'+X['dt_of_fr'].str.slice_replace(start=7, stop=20, repl='').str.replace(pat='-', repl='', regex=False)
            ) # gas/ele + 불이 난 달을 값으로 갖는 열을 생성합니다. 이걸 생성하지 않고 하는 것도 할 수 있을거 같은데 지금은 뇌가 멈췄으니 일단 만듭니다.
    
    tk = train_rst.keys()[np.where(pd.Series(train_rst.keys()) == index[0])[0][0]:np.where(pd.Series(train_rst.keys()) == index[-1])[0][0]+1]
    keys = pd.Series(tk).str.split('_', expand = True).iloc[:,[0,3]]
    keys[1] = keys[0]+keys[3]
    keys[1][0] = 'dt_of_fr' # 변경된 열 이름의 리스트입니다. 가장 앞의 값이 누락되어서 추가해주었습니다.
    
    coln = keys[1].ravel().tolist()+['fr_month_gas','fr_month_ele'] #열 이름을 바꿔줄 준비. 앞선 fr_month_gas/ ele열을 index로 사용하기 위함입니다
    train_rst.columns = coln #열 이름 변경합니다.

    fm_gas_index = train_rst.fr_month_gas.map(lambda x: np.where(x == train_rst.keys())[0][0]) # 불이 난 월의 gas의 열 index
    fm_ele_index = train_rst.fr_month_ele.map(lambda x: np.where(x == train_rst.keys())[0][0]) # 불이 난 월의 ele의 열 index

    train_eg = train_rst.assign(
        gas_fm = np.array(train_rst)[np.arange(validation.shape[0]),fm_gas_index.values] # 최종적으로 두 index를 기준으로 값을 대입합니다.
        ).assign(
            ele_fm = np.array(train_rst)[np.arange(validation.shape[0]),fm_ele_index.values]
            )[['gas_fm','ele_fm']] # 이 두 열만 남기거나
            #.drop(['fr_month_gas','fr_month_ele','dt_of_fr'], axis = 1) # 이 두 열을 더하거나. 일단은 두 열만 남겼습니다. 
    
    return np.array(train_eg)

In [112]:
# Test 

class TeFmGEAdder(BaseEstimator, TransformerMixin):
  def __init__(self, index = eg_var):
    self.index = ['dt_of_fr'] + index
  
  def fit(self, X,y = None):
    return(self)
  
  def transform(self, X, y = None):
    index = self.index
    X = np.append(np.array(test['dt_of_fr']).reshape(-1,1), X, axis = 1) # X를 받아서 앞에 dt_of_fr열을 추가합니다.
    X = pd.DataFrame(X) # 차후 열 이름을 기준으로 사용해야 하기 때문에 Pandas로 변환합니다. 
    X.columns = test[['dt_of_fr']+eg_var].keys() # 열 이름을 추가합니다. eg_var 대상으로 진행할 것이기 때문에 이렇게 씁니다.

    train_rst = X.assign(
        fr_month_gas  = 'gas'+X['dt_of_fr'].str.slice_replace(start=7, stop=20, repl='').str.replace(pat='-', repl='', regex=False)
        ).assign(
            fr_month_ele  = 'ele'+X['dt_of_fr'].str.slice_replace(start=7, stop=20, repl='').str.replace(pat='-', repl='', regex=False)
            ) # gas/ele + 불이 난 달을 값으로 갖는 열을 생성합니다. 이걸 생성하지 않고 하는 것도 할 수 있을거 같은데 지금은 뇌가 멈췄으니 일단 만듭니다.
    
    tk = train_rst.keys()[np.where(pd.Series(train_rst.keys()) == index[0])[0][0]:np.where(pd.Series(train_rst.keys()) == index[-1])[0][0]+1]
    keys = pd.Series(tk).str.split('_', expand = True).iloc[:,[0,3]]
    keys[1] = keys[0]+keys[3]
    keys[1][0] = 'dt_of_fr' # 변경된 열 이름의 리스트입니다. 가장 앞의 값이 누락되어서 추가해주었습니다.
    
    coln = keys[1].ravel().tolist()+['fr_month_gas','fr_month_ele'] #열 이름을 바꿔줄 준비. 앞선 fr_month_gas/ ele열을 index로 사용하기 위함입니다
    train_rst.columns = coln #열 이름 변경합니다.

    fm_gas_index = train_rst.fr_month_gas.map(lambda x: np.where(x == train_rst.keys())[0][0]) # 불이 난 월의 gas의 열 index
    fm_ele_index = train_rst.fr_month_ele.map(lambda x: np.where(x == train_rst.keys())[0][0]) # 불이 난 월의 ele의 열 index

    train_eg = train_rst.assign(
        gas_fm = np.array(train_rst)[np.arange(test.shape[0]),fm_gas_index.values] # 최종적으로 두 index를 기준으로 값을 대입합니다.
        ).assign(
            ele_fm = np.array(train_rst)[np.arange(test.shape[0]),fm_ele_index.values]
            )[['gas_fm','ele_fm']] # 이 두 열만 남기거나
            #.drop(['fr_month_gas','fr_month_ele','dt_of_fr'], axis = 1) # 이 두 열을 더하거나. 일단은 두 열만 남겼습니다. 
    
    return np.array(train_eg)

In [113]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# 왜인지는 모르겠는데 full_pipeline을 만들 때 ColumnTransformer는 자꾸 에러가 나서 FeatureUnion을 쓰려고 DataFramseSelector를 정의해 주었습니다.

## urban_population_feature Pipeline

In [114]:
urban_transformer = Pipeline(steps = [
    ('constant', SimpleImputer(strategy = 'most_frequent')),
    ('OneHotEncoer', OneHotEncoder())
])

numerical_transformer = Pipeline([
    ('median', SimpleImputer(strategy = 'median')),
    ('scaling', StandardScaler())
])

price_transformer = Pipeline(steps = [
    ('median', SimpleImputer(strategy = 'median')),
    ('scaling', StandardScaler()),
    ('pca', PCA()),
])

processor_1 = ColumnTransformer(
    transformers = [
    ('urb', urban_transformer, urban_feature),
    ('num', numerical_transformer, numerical_feature),
    ('price', price_transformer, price_feature)
])

## building_feature Pipeline

In [115]:
building_numeric_transformer = Pipeline(steps = [
    ('scaling', StandardScaler()),
    ('pca', PCA()),
])


building_date_transformer = Pipeline([
    ('imp', SimpleImputer(strategy = 'constant', fill_value = 0))
])


processor_2 = ColumnTransformer(
    transformers = [
    ('bldng_num', building_numeric_transformer, building_numeric_feature),
    ('bldng_date', building_date_transformer, building_date)
])

---------
---------

In [116]:
#Train
Treg_imputer = Pipeline([
                       ('selector', DataFrameSelector(eg_var)),
                       ('zero_imputer', SimpleImputer(strategy='constant',fill_value=0)),
                       ('scaler', StandardScaler()),
                       ('fire month gas ele adder', TrFmGEAdder())
])

lw_imputer = Pipeline([
                      ('selector', DataFrameSelector(lw_var)),
                      ('zero_imputer', SimpleImputer(strategy='constant', fill_value=0)),
                      ('scaler', StandardScaler())
])
'''
Tregl_pipe = FeatureUnion(transformer_list = [
                                            ('eg_imputer', Treg_imputer),
                                            ('lw_imputer', lw_imputer)
                     ])
'''

"\nTregl_pipe = FeatureUnion(transformer_list = [\n                                            ('eg_imputer', Treg_imputer),\n                                            ('lw_imputer', lw_imputer)\n                     ])\n"

In [117]:
#Validation
Veg_imputer = Pipeline([
                       ('selector', DataFrameSelector(eg_var)),
                       ('zero_imputer', SimpleImputer(strategy='constant',fill_value=0)),
                       ('scaler', StandardScaler()),
                       ('fire month gas ele adder', VFmGEAdder())
])

lw_imputer = Pipeline([
                      ('selector', DataFrameSelector(lw_var)),
                      ('zero_imputer', SimpleImputer(strategy='constant', fill_value=0)),
                      ('scaler', StandardScaler())
])
'''
Vegl_pipe = FeatureUnion(transformer_list = [
                                            ('eg_imputer', Veg_imputer),
                                            ('lw_imputer', lw_imputer)
                     ])
'''

"\nVegl_pipe = FeatureUnion(transformer_list = [\n                                            ('eg_imputer', Veg_imputer),\n                                            ('lw_imputer', lw_imputer)\n                     ])\n"

In [118]:
#Test
Teeg_imputer = Pipeline([
                       ('selector', DataFrameSelector(eg_var)),
                       ('zero_imputer', SimpleImputer(strategy='constant',fill_value=0)),
                       ('scaler', StandardScaler()),
                       ('fire month gas ele adder', TeFmGEAdder())
])

lw_imputer = Pipeline([
                      ('selector', DataFrameSelector(lw_var)),
                      ('zero_imputer', SimpleImputer(strategy='constant', fill_value=0)),
                      ('scaler', StandardScaler())
])
'''
Teegl_pipe = FeatureUnion(transformer_list = [
                                            ('eg_imputer', Teeg_imputer),
                                            ('lw_imputer', lw_imputer)
                     ])
'''

"\nTeegl_pipe = FeatureUnion(transformer_list = [\n                                            ('eg_imputer', Teeg_imputer),\n                                            ('lw_imputer', lw_imputer)\n                     ])\n"

# 최종적으로 모델에 들어가는 모양

In [119]:
Train_full_pipe = FeatureUnion(transformer_list = [
    ('weather',weather_pipe ),
    ('eg_imputer', Treg_imputer),
    ('lw_impute', lw_imputer),
    ('processor_1', processor_1),
    ('processor_2', processor_2)

])

In [120]:
Validation_full_pipe = FeatureUnion(transformer_list = [
    ('weather',weather_pipe ),
    ('eg_imputer', Veg_imputer),
    ('lw_impute', lw_imputer),
    ('processor_1', processor_1),
    ('processor_2', processor_2)
    
])

In [121]:
Test_full_pipe = FeatureUnion(transformer_list = [
    ('weather',weather_pipe ),
    ('eg_imputer', Teeg_imputer),
    ('lw_impute', lw_imputer),
    ('processor_1', processor_1),
    ('processor_2', processor_2)
    
])

# Base Model

In [122]:
Train_X = Train_full_pipe.fit_transform(train_2)
Validation_X = Validation_full_pipe.fit_transform(validation_2)
Test_X = Test_full_pipe.fit_transform(test_2)

In [123]:
Train_X = np.c_[Train_X,train_3]
Validation_X = np.c_[Validation_X,validation_3]
Test_X = np.c_[Test_X,test_3]

In [124]:
Train_y = np.array(train.fr_yn).reshape(-1,1)
Validation_y = np.array(validation.fr_yn).reshape(-1,1)

In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [126]:
RF_clf = RandomForestClassifier(min_samples_leaf=5)



In [127]:
RF_clf.fit(Train_X,Train_y)


/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [128]:
sum(RF_clf.predict(Train_X)==Train_y.reshape(-1))/len(Train_X)

0.9229209952870826

In [129]:
sum(RF_clf.predict(Validation_X)==Validation_y.reshape(-1))/len(Validation_X)

0.8237170194259206

In [130]:
f1_score(Validation_y.reshape(-1), RF_clf.predict(Validation_X))



0.3926073926073926

In [145]:
from sklearn.base import clone


In [158]:
max_f_for_fun = 0.41
for i in range(200) :
    RF_clf.fit(Train_X,Train_y)
    new_score = f1_score(Validation_y.reshape(-1), RF_clf.predict(Validation_X))
    if new_score > max_f_for_fun :
        train_f_score = f1_score(Train_y.reshape(-1), RF_clf.predict(Train_X))
        max_f_for_fun = new_score
        submission = RF_clf.predict(Test_X)


/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ieunpyo/anaconda3/

In [159]:
print(train_f_score)
print(max_f_for_fun)

0.5926120448179272
0.4117359413202934


# HYPERPARAMETER TUNING

## Random Hyperparameter Grid


In [165]:
RF_clf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [166]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [168]:
random_grid

{'n_estimators': [10, 231, 452, 673, 894, 1115, 1336, 1557, 1778, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [169]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf_clf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_clf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)


In [170]:
# Fit the random search model
rf_random.fit(Train_X, Train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 144.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 698.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 855.9min finished
/Users/ieunpyo/anaconda3/envs/work/lib/python3.7/site-packages/sklearn/model_selection/_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [171]:
rf_random.best_params_

{'n_estimators': 894,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': False}

In [172]:
f1_score(Validation_y.reshape(-1),rf_random.predict(Validation_X))

0.3565166569257744

In [173]:
f1_score(Train_y.reshape(-1), RF_clf.predict(Train_X))

0.6108577819839972

In [ ]:
#submission = RF_clf.predict(Test_X)
submission = pd.DataFrame(submission,columns=['fr_yn'])
submission['fr_yn'] = submission['fr_yn'].map({1:'Y', 0:'N'})

In [ ]:
submission.to_csv('화재예측과제_Submission.csv')

In [ ]:
train['urban']